## 2.2 HTML to text conversion

In [ ]:
from resiliparse.parse.encoding import detect_encoding
from resiliparse.extract.html2text import extract_plain_text

def run_extract_text_from_html_bytes(html_bytes: bytes) -> str | None:

    # Detect encoding of the byte string
    enc = detect_encoding(html_bytes)
    print(enc)

    # Decode the byte string into a Unicode string
    html = html_bytes.decode('utf-8')
    print(html)

    # If the encoding is not UTF-8, try to decode it using the detected encoding
    if enc != 'utf-8':
        try:
            html = html_bytes.decode(enc)
        except UnicodeDecodeError:
            return None

    # Extract text from the HTML string
    text = extract_plain_text(html)

    return text

In [17]:
# run_extract_text_from_html_bytes(b'\xff\xfeH\x00e\x00l\x00l\x00o\x00 \x00W\x00o\x00r\x00l\x00d\x00')
run_extract_text_from_html_bytes(b'<html><head><title>Test</title></head><body><h1>Hello World</h1></body></html>')

cp1252
<html><head><title>Test</title></head><body><h1>Hello World</h1></body></html>


'Hello World'